# Домашнее задание к уроку 3


Задача поиск похожих по эмбеддингам

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать 
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов

на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

3. Проверить насколько хорошо работают подходы

Подготовим данные

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [3]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [4]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [5]:
df.shape

(226834, 2)

In [56]:
import codecs
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm as tqdm_notebook


In [19]:
fin = codecs.open("tweets.txt", "w", "utf_8_sig" )
for i in df['text']:
    fin.write(i)
fin.close()

In [29]:
assert True

fout = codecs.open("prepared.txt", "w", "utf_8_sig" )
fin = codecs.open("tweets.txt", "r", "utf_8_sig" )
for line in tqdm_notebook(fin):
    if line != "\n":
        fout.write(line.replace("\t", " ").replace("\n", " ") + "\t" + line.replace("\t", " "))
fout.close()
fin.close()

In [30]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [31]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
fin = codecs.open("tweets.txt","r","utf_8_sig")
for line in tqdm_notebook(fin):
    spls = preprocess_txt(line)
    sentences.append(spls)

In [32]:
sentences = [i for i in sentences if len(i) > 2]

In [34]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=2)

In [35]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=2, window=5, workers=8)

In [37]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0
f = codecs.open("prepared.txt", "r", "utf_8_sig" )
for line in tqdm_notebook(f):
    n_w2v = 0
    n_ft = 0
    spls = line.split("\t")
    index_map[counter] = spls[1]
    question = preprocess_txt(spls[0])

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [38]:
def get_response(question, index, model, index_map,N):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, N, )
    return [index_map[i] for i in answers]

In [69]:
TEXT = "погуляли и пошли потом домой"

In [70]:
get_response(TEXT, w2v_index, modelW2V, index_map,5)

['Домой хочу безумно(Расшифровываю интервью....И теперь там ничего не слышно :( #безысходность #тлен \n',
 'отпустили домой с урока)@mushka_99 ну не красиво просто))\n',
 'как твой?Обожаю ездить домой с хабзой,прям тащусь.,!)\n',
 ' ужасно погуляла, приехала домой.. родители наорали..\n',
 'На бис)Завтра домой)) мамуля сказала что всего напокупала и ждет))RT @ollivercom: @MadDirectioner1 пхахп, художник :-)\n']

In [71]:
get_response(TEXT, ft_index, modelFT, index_map,5)

[' ужасно погуляла, приехала домой.. родители наорали..\n',
 'пусть Триода едут домой\n',
 'продолжил по пути домой\n',
 'Она не отпускает меня домой ска((99(((@nata19991tasha ну да, они такие((( \n',
 'http://t.co/kAb5dsgWda@os_darya ДАВАЙ ПОБЕГАЕМ ПО ДОМУ!\n']

In [ ]:
import gensim.downloader as api

In [64]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [72]:


word_vectors = api.load("glove-wiki-gigaword-300")  # загрузим предтренированные вектора слов из gensim-data


Не нашел модели GloVe. Нашел что только можно загрузить предтренированные вектора, но и то только на английском.

Подскажите, пожалуста, как использовать glove